In [3]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask_ml.model_selection import train_test_split

In [5]:
# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv('./data/categories_string.csv')['0'].to_dict()


# Importation du jeu de données d'entrainement

trainX=dd.from_pandas(pd.read_json('./data/train.json',dtype={'description':'string','gender':'string'})
         .set_index('Id'), npartitions=11)

trainY=dd.read_csv('./data/train_label.csv')
         .set_index('Id')

# Concatenation en un seul DataFrame pour visualiser
trainDF= trainX.assign(Y = trainY.Category).categorize(columns=['gender'])

In [ ]:
print(jobNames)

In [ ]:
trainDF.head()

In [ ]:
def macro_disparate_impact(DF):
    """Fonction Dask de calcul de la Fairness (disparate_impact)"""
    # "Jointure" sur les noms de métiers
    jobs=DF.Y.apply(lambda x :jobNames[x],meta=('Y', 'int'))
    
    # Concatenation métier | genre
    people=dd.concat([jobs,DF.gender], axis=1).reset_index()
    
    # Création de la table de comptage par genre
    counts = people.pivot_table(index='Y',columns='gender',aggfunc='count')
    
    # Calcul de l'indicateur
    # Paramètre axis=1 <== 'max/min par ligne'
    disparate_impact= (counts.max(axis=1)/counts.min(axis=1)).mean()
    
    return disparate_impact

In [ ]:
macro_disparate_impact(trainDF).compute()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from dask_ml.feature_extraction.text import HashingVectorizer

In [ ]:
trainDF['description']=trainDF.apply(lambda x:x['description'].lower(),axis=1,meta=(None, 'object'))

In [ ]:
trainDF1.head()

In [ ]:
trainSet,validSet=train_test_split(trainDF,shuffle=True,test_size=0.3)

In [ ]:


transformer = TfidfVectorizer().fit(train_df["description_lower"].values)

print("NB features: %d" %(len(transformer.vocabulary_)))

X_train = transformer.transform(train_df["description_lower"].values)

X_test = transformer.transform(test_df["description_lower"].values)

X_train